In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rd-kit/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train.csv
/kaggle/input/neurips-open-polymer-prediction-2025/test.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv


In [75]:
!pip install /kaggle/input/rd-kit/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/rd-kit/rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
rdkit-pypi is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [76]:
import pandas as pd
train_csv = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train.csv")
train_csv['SMILES'][1]

'*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5ccc(N*)cc5)cc4)CCC(CCCCC)CC3)cc2)cc1'

In [77]:
test_csv = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/test.csv")
test_csv

,id,SMILES
0,1109053969,*Oc1ccc(C=NN=Cc2ccc(Oc3ccc(C(c4ccc(*)cc4)(C(F)...
1,1422188626,*Oc1ccc(C(C)(C)c2ccc(Oc3ccc(C(=O)c4cccc(C(=O)c...
2,2032016830,*c1cccc(OCCCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6...


In [78]:
Tc_extra = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv")
test2 = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv")
Tg_extra = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv")
FFV_extra = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv")

Tc_extra.rename(columns={"TC_mean": "Tc"}, inplace=True)
Tc_extra

,SMILES,Tc
0,*/C(=C(\c1ccccc1)c1ccc(*)cc1)c1ccccc1,0.3380
1,*/C(F)=C(\F)C(F)(C(*)(F)F)C(F)(F)F,0.1020
2,*/C=C(/*)C#CCCCCCCCCCCCCCCCCCCCCC(=O)O,0.4105
3,*/C=C(/*)CCCCCCCCCCCCCCCCCCCCC(=O)O,0.4030
4,*/C=C/*,0.5260
...,...,...
869,CCNC(=O)OCCCCC(*)=C=C=C(*)CCCCOC(=O)NCC,0.2600
870,Cc1ccc(cc1)S(=O)(=O)OCCCCC(*)=C=C=C(*)CCCCOS(=...,0.2100
871,FC(F)(F)C(C1=CC2=C(OC(=N2)C2=CC=C(OC3=C4C5C6=C...,0.1090
872,FC(F)(F)C(C1=CC2=C(OC(=N2)C2=CC=C(OC3=C4C5C6=C...,0.1760


In [79]:
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from rdkit import DataStructs
import numpy as np

def smiles_to_features(smiles, fp_size=1024, radius=2):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        # Return zeros if invalid molecule
        return np.zeros(fp_size + 7, dtype=float)
    
    # 1. Descriptors
    desc_list = [
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Descriptors.TPSA(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.RingCount(mol),
    ]
    
    # 2. Fingerprint
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=fp_size)
    fp_arr = np.zeros((fp_size,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, fp_arr)
    
    # 3. Concatenate
    return np.concatenate([desc_list, fp_arr])


In [80]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, DataStructs
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


In [81]:
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/new-tokenizer")


In [82]:

# input_text = 'C1=CC2=C(C(=C1)[O-])NC(=CC2=O)C(=O)O'
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids


In [83]:
# input_text = train_csv['SMILES'][1]
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids
# input_ids
#outputs = model.generate(input_ids, num_beams=5, max_length=512)
#print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [84]:
# input_ids.shape

In [85]:
non_null_count = train_csv['FFV'].notnull().sum()
print(non_null_count) 

7030


In [86]:
features = ['SMILES','FFV']
FFV_train_ = train_csv[train_csv['FFV'].notnull()][features].copy()

features = ['SMILES','Tg']
Tg_train_ = train_csv[train_csv['Tg'].notnull()][features].copy()

features = ['SMILES','Tc']
Tc_train_ = train_csv[train_csv['Tc'].notnull()][features].copy()

features = ['SMILES','Density']
Density_train = train_csv[train_csv['Density'].notnull()][features].copy()

features = ['SMILES','Rg']
Rg_train = train_csv[train_csv['Rg'].notnull()][features].copy()


In [87]:
Tc_train_

,SMILES,Tc
0,*CC(*)c1ccccc1C(=O)OCCCCCC,0.205667
10,*c1ccc(-c2ccc3c(c2)C(CCCCCCC#N)(CCCCCCC#N)c2cc...,0.487000
11,*CC(*)c1ccc(C(=O)O)c(C(=O)O)c1,0.171000
31,*CCCCCNC(=O)CCCCC(=O)N*,0.327000
34,*CCCCCCCCCCCCCCCCCCNC(=O)NCCCCCCNC(=O)N*,0.383000
...,...,...
7899,*CCN(*)C(=O)CCCCCCCCCCCCCCC,0.336000
7905,*CC(*)c1ccccc1F,0.174667
7943,*CC(*)C1CC=CCC1,0.180750
7944,*CC(*)Cl,0.139000


In [88]:
FFV_train = pd.concat([FFV_train_, FFV_extra], ignore_index = True)
Tg_train = pd.concat([Tg_train_, Tg_extra], ignore_index = True)
Tc_train = pd.concat([Tc_train_, Tc_extra], ignore_index = True)
print(FFV_train.shape)
print(Tg_train.shape)
print(Tc_train.shape)
print(Density_train.shape)
print(Rg_train.shape)

(7892, 2)
(557, 2)
(1611, 2)
(613, 2)
(614, 2)


In [89]:
# Example: build X and y
X = []
y = []

for smi, target in zip(FFV_train["SMILES"], FFV_train["FFV"]):
    feats = smiles_to_features(smi)
    if feats is not None:
        X.append(feats)
        y.append(target)

X = np.array(X)
y = np.array(y)

print("Feature matrix shape:", X.shape)


Feature matrix shape: (7892, 1031)


In [90]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras import regularizers

In [91]:
# model_FFV = Sequential([
#     Dense(150, activation='relu', input_shape=(75,)),
#     Dropout(0.3),
#     Dense(75, activation='relu'),
#     Dropout(0.3),
#     Dense(1, activation='linear')
# ])

In [92]:
# X = tokenizer(
#     FFV_train["SMILES"].tolist(),
#     padding="max_length",
#     truncation=True,
#     max_length=75,  
#     return_tensors="np" 
# ).input_ids

# print(X.shape)

In [93]:
from sklearn.model_selection import train_test_split
Y = FFV_train['FFV'].values
X_train_FFV, X_test_FFV, Y_train_FFV, Y_test_FFV = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

In [94]:
# model_FFV.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#              loss = "mae",                  
#     metrics=['mae','mse','mape'])

In [95]:
# import tensorflow as tf

# model_FFV.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#     loss="mae",                 
#     metrics=["mae"]             
# )
# sample_weights = np.ones(len(X_train))
#
# history = model_FFV.fit(
#     X_train, Y_train,
#     sample_weight=sample_weights,   
#     validation_data=(X_test, Y_test),
#     epochs=100,
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(
#             monitor="val_mae", mode="min", patience=20, restore_best_weights=True
#         ),
#         tf.keras.callbacks.ReduceLROnPlateau(
#             monitor="val_mae", mode="min", factor=0.5, patience=4
#         )
#     ],
#     batch_size=32
# )


In [96]:
#Trying XGBOOST
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
model_FFV = xgb.XGBRegressor(
    n_estimators=3300,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1,    
    reg_alpha=0.1,   
    random_state=42,
    tree_method="hist"
)

In [97]:
model_FFV.fit(
    X_train_FFV, Y_train_FFV,
    eval_set=[(X_test_FFV, Y_test_FFV)],
    eval_metric="mae",
    early_stopping_rounds=50,
    verbose=50
)
Y_pred_FFV = model_FFV.predict(X_test_FFV)
print("Validation MAE:", mean_absolute_error(Y_test_FFV, Y_pred_FFV))

[0]	validation_0-mae:0.02001


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-mae:0.00897
[100]	validation_0-mae:0.00758
[150]	validation_0-mae:0.00705
[200]	validation_0-mae:0.00675
[250]	validation_0-mae:0.00657
[300]	validation_0-mae:0.00642
[350]	validation_0-mae:0.00633
[400]	validation_0-mae:0.00625
[450]	validation_0-mae:0.00620
[500]	validation_0-mae:0.00615
[550]	validation_0-mae:0.00611
[600]	validation_0-mae:0.00608
[650]	validation_0-mae:0.00606
[700]	validation_0-mae:0.00603
[750]	validation_0-mae:0.00601
[800]	validation_0-mae:0.00599
[850]	validation_0-mae:0.00599
[900]	validation_0-mae:0.00597
[950]	validation_0-mae:0.00597
[1000]	validation_0-mae:0.00596
[1050]	validation_0-mae:0.00595
[1100]	validation_0-mae:0.00594
[1150]	validation_0-mae:0.00593
[1200]	validation_0-mae:0.00592
[1250]	validation_0-mae:0.00591
[1300]	validation_0-mae:0.00591
[1350]	validation_0-mae:0.00591
[1400]	validation_0-mae:0.00590
[1450]	validation_0-mae:0.00589
[1500]	validation_0-mae:0.00589
[1550]	validation_0-mae:0.00588
[1600]	validation_0-mae:0.00

In [98]:
#model_FFV.fit(X_train, Y_train, epochs = 50, batch_size=32)

In [99]:
# results = model_FFV.evaluate(X_test, Y_test)
# print(f"Test loss: {results[0]}")
# print(f"Test MAE: {results[1]}")
#print(f"Test MSE: {results[2]}")
##print(f"Test MAPE: {results[3]}")

In [100]:

# X_sub = tokenizer(
#     test_csv["SMILES"].tolist(),
#     padding="max_length",
#     truncation=True,
#     max_length=75,  
#     return_tensors="np"  
# ).input_ids

# Y_FFV_sub = model_FFV.predict(X_sub)

In [101]:
# Example: build X and y
X_sub = []
#y = []

for smi in test_csv["SMILES"]:
    feats = smiles_to_features(smi)
    if feats is not None:
        X_sub.append(feats)

X_sub = np.array(X_sub)

print("sub matrix shape:", X_sub.shape)

sub matrix shape: (3, 1031)


In [102]:
Y_FFV_sub = model_FFV.predict(X_sub)
Y_FFV_sub

array([0.3683966 , 0.37637457, 0.35037133], dtype=float32)

In [103]:
# model_Tg = Sequential([
#     Dense(75, activation='relu', input_shape=(75,), kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
#     Dropout(0.3),
#     Dense(50, activation='relu'),
#     Dense(1, activation='linear')
# ])

# X_Tg = tokenizer(
#     Tg_train["SMILES"].tolist(),
#     padding="max_length",
#     truncation=True,
#     max_length=75,  
#     return_tensors="np"  
# ).input_ids

# print(X_Tg.shape)

# model_Tg.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#              loss = "mae",                  
#     metrics=['mae','mse','mape'])

In [104]:
X_Tg = []
Y_Tg = []

for smi, target in zip(Tg_train["SMILES"], Tg_train["Tg"]):
    feats = smiles_to_features(smi)
    if feats is not None:
        X_Tg.append(feats)
        Y_Tg.append(target)

X_Tg = np.array(X_Tg)
Y_Tg = np.array(Y_Tg)

print("Feature matrix shape:", X_Tg.shape)

Feature matrix shape: (557, 1031)


In [105]:
#Trying XGBOOST
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
model_Tg = xgb.XGBRegressor(
    n_estimators=1400,
    learning_rate=0.04,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1,    
    reg_alpha=0.1,   
    random_state=42,
    tree_method="hist"
)
# X_Tg = tokenizer(
#     Tg_train["SMILES"].tolist(),
#     padding="max_length",
#     truncation=True,
#     max_length=75,  
#     return_tensors="np"  
# ).input_ids
Y_Tg = Tg_train['Tg'].values
X_train_Tg, X_test_Tg, Y_train_Tg, Y_test_Tg = train_test_split(X_Tg, Y_Tg, test_size=0.2, random_state=42,shuffle=True)


In [106]:
model_Tg.fit(
    X_train_Tg, Y_train_Tg,
    eval_set=[(X_test_Tg, Y_test_Tg)],
    eval_metric="mae",
    early_stopping_rounds=50,
    verbose=50
)

[0]	validation_0-mae:82.90057


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-mae:56.24296
[100]	validation_0-mae:55.05285
[126]	validation_0-mae:54.95870


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.04, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1400, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [107]:
Y_pred_Tg = model_Tg.predict(X_test_Tg)
print("Validation MAE:", mean_absolute_error(Y_test_Tg, Y_pred_Tg))

Validation MAE: 54.770116905646375


In [108]:
# model_Tc = Sequential([
#     Dense(150, activation='relu', input_shape=(75,)),
#     Dropout(0.3),
#     Dense(75, activation='relu'),
#     Dropout(0.3),
#     Dense(1, activation='linear')
# ])

# X_Tc = tokenizer(
#     Tc_train["SMILES"].tolist(),
#     padding="max_length",
#     truncation=True,
#     max_length=75,  
#     return_tensors="np"  
# ).input_ids

# print(X_Tc.shape)

# model_Tc.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#              loss = "mae",                  
#     metrics=['mae','mse','mape'])

In [109]:
X_Tc = []
Y_Tc = []

for smi, target in zip(Tc_train["SMILES"], Tc_train["Tc"]):
    feats = smiles_to_features(smi)
    if feats is not None:
        X_Tc.append(feats)
        Y_Tc.append(target)

X_Tc = np.array(X_Tc)
Y_Tc = np.array(Y_Tc)

print("Feature matrix shape:", X_Tc.shape)

Feature matrix shape: (1611, 1031)


In [110]:
#Trying XGBOOST

model_Tc = xgb.XGBRegressor(
    n_estimators=2700,
    learning_rate=0.04,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1,    
    reg_alpha=0.1,   
    random_state=42,
    tree_method="hist"
)
# X_Tc = tokenizer(
#     Tc_train["SMILES"].tolist(),
#     padding="max_length",
#     truncation=True,
#     max_length=75,  
#     return_tensors="np"  
# ).input_ids

Y_Tc = Tc_train['Tc'].values
X_train_Tc, X_test_Tc, Y_train_Tc, Y_test_Tc = train_test_split(X_Tc, Y_Tc, test_size=0.2, random_state=42,shuffle=True)


In [111]:
model_Tc.fit(
    X_train_Tc, Y_train_Tc,
    eval_set=[(X_test_Tc, Y_test_Tc)],
    eval_metric="mae",
    early_stopping_rounds=50,
    verbose=50
)
Y_pred_Tc = model_Tc.predict(X_test_Tc)
print("Validation MAE:", mean_absolute_error(Y_test_Tc, Y_pred_Tc))

[0]	validation_0-mae:0.07223


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-mae:0.02742
[100]	validation_0-mae:0.02346
[150]	validation_0-mae:0.02171
[200]	validation_0-mae:0.02048
[250]	validation_0-mae:0.01964
[300]	validation_0-mae:0.01897
[350]	validation_0-mae:0.01845
[400]	validation_0-mae:0.01802
[450]	validation_0-mae:0.01764
[500]	validation_0-mae:0.01738
[550]	validation_0-mae:0.01714
[600]	validation_0-mae:0.01692
[650]	validation_0-mae:0.01676
[700]	validation_0-mae:0.01660
[750]	validation_0-mae:0.01642
[800]	validation_0-mae:0.01630
[850]	validation_0-mae:0.01617
[900]	validation_0-mae:0.01608
[950]	validation_0-mae:0.01598
[1000]	validation_0-mae:0.01588
[1050]	validation_0-mae:0.01580
[1100]	validation_0-mae:0.01570
[1150]	validation_0-mae:0.01565
[1200]	validation_0-mae:0.01556
[1250]	validation_0-mae:0.01549
[1300]	validation_0-mae:0.01542
[1350]	validation_0-mae:0.01535
[1400]	validation_0-mae:0.01529
[1450]	validation_0-mae:0.01524
[1500]	validation_0-mae:0.01519
[1550]	validation_0-mae:0.01515
[1600]	validation_0-mae:0.01

In [112]:
# model_Density = Sequential([
#     Dense(75, activation='relu', input_shape=(75,)),
#     Dropout(0.3),
#     Dense(50, activation = 'relu'),
#     Dropout(0.3),
#     Dense(1, activation='linear')
# ])

# X_Density = tokenizer(
#     Density_train["SMILES"].tolist(),
#     padding="max_length",
#     truncation=True,
#     max_length=75,  
#     return_tensors="np"  
# ).input_ids

# print(X_Density.shape)


# model_Density.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#              loss = "mae",                  
#     metrics=['mae','mse','mape'])

In [113]:
X_Density = []
Y_Density = []

for smi, target in zip(Density_train["SMILES"], Density_train["Density"]):
    feats = smiles_to_features(smi)
    if feats is not None:
        X_Density.append(feats)
        Y_Density.append(target)

X_Density = np.array(X_Density)
Y_Density = np.array(Y_Density)

print("Feature matrix shape:", X_Density.shape)

Feature matrix shape: (613, 1031)


In [114]:
# XGBOOST
model_Density = xgb.XGBRegressor(
    n_estimators=1500,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.6,
    colsample_bytree=0.8,
    reg_lambda=1,    
    reg_alpha=0.1,   
    random_state=42,
    tree_method="hist"
)
# X_Density = tokenizer(
#     Density_train["SMILES"].tolist(),
#     padding="max_length",
#     truncation=True,
#     max_length=75,  
#     return_tensors="np"  
# ).input_ids
Y_Density = Density_train['Density'].values
X_train_D, X_test_D, Y_train_D, Y_test_D = train_test_split(X_Density, Y_Density, test_size=0.2, random_state=42,shuffle=True)

In [115]:
model_Density.fit(
    X_train_D, Y_train_D,
    eval_set=[(X_test_D, Y_test_D)],
    eval_metric="mae",
    early_stopping_rounds=50,
    verbose=50
)
Y_pred_D = model_Density.predict(X_test_D)
print("Validation MAE:", mean_absolute_error(Y_test_D, Y_pred_D))

[0]	validation_0-mae:0.10333


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-mae:0.04521
[100]	validation_0-mae:0.03771
[150]	validation_0-mae:0.03563
[200]	validation_0-mae:0.03490
[250]	validation_0-mae:0.03459
[300]	validation_0-mae:0.03419
[350]	validation_0-mae:0.03392
[400]	validation_0-mae:0.03387
[420]	validation_0-mae:0.03386
Validation MAE: 0.033787475605970424


In [116]:
# model_Rg = Sequential([
#     Dense(75, activation='relu', input_shape=(75,)),
#     Dropout(0.3),
#     Dense(45, activation='relu'),
#     Dropout(0.3),
#     Dense(1, activation='linear')
# ])

# X_Rg = tokenizer(
#     Rg_train["SMILES"].tolist(),
#     padding="max_length",
#     truncation=True,
#     max_length=75,  
#     return_tensors="np"  
# ).input_ids

# print(X_Rg.shape)


# model_Rg.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#              loss = "mae",                  
#     metrics=['mae','mse','mape'])

In [117]:
X_Rg = []
Y_Rg = []

for smi, target in zip(Rg_train["SMILES"], Rg_train["Rg"]):
    feats = smiles_to_features(smi)
    if feats is not None:
        X_Rg.append(feats)
        Y_Rg.append(target)

X_Rg = np.array(X_Rg)
Y_Rg = np.array(Y_Rg)

print("Feature matrix shape:", X_Rg.shape)

Feature matrix shape: (614, 1031)


In [118]:
# XGBOOST
model_Rg = xgb.XGBRegressor(
    n_estimators=1700,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1,    
    reg_alpha=0.1,   
    random_state=42,
    tree_method="hist"
)
# X_Rg = tokenizer(
#     Rg_train["SMILES"].tolist(),
#     padding="max_length",
#     truncation=True,
#     max_length=75,  
#     return_tensors="np"  
# ).input_ids
Y_Rg = Rg_train['Rg'].values
X_train_Rg, X_test_Rg, Y_train_Rg, Y_test_Rg = train_test_split(X_Rg, Y_Rg, test_size=0.2, random_state=42,shuffle=True)

In [119]:
model_Rg.fit(
    X_train_Rg, Y_train_Rg,
    eval_set=[(X_test_Rg, Y_test_Rg)],
    eval_metric="mae",
    early_stopping_rounds=50,
    verbose=50
)
Y_pred_Rg = model_Rg.predict(X_test_Rg)
print("Validation MAE:", mean_absolute_error(Y_test_Rg, Y_pred_Rg))

[0]	validation_0-mae:3.76726
[50]	validation_0-mae:1.71029


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[100]	validation_0-mae:1.63332
[150]	validation_0-mae:1.64338
[156]	validation_0-mae:1.64733
Validation MAE: 1.6288184109909372


In [120]:
# model_Tg.fit(X_train_Tg, Y_train_Tg, epochs=30, batch_size=32)

In [121]:
# history = model_Tg.fit(
#     X_train_Tg, Y_train_Tg,
#     validation_data=(X_test_Tg, Y_test_Tg),
#     epochs=200,
#     batch_size=16,   # smaller batch since small dataset
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=20, restore_best_weights=True),
#         tf.keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=5)
#     ]
# )

In [122]:
# results = model_Tg.evaluate(X_test_Tg, Y_test_Tg)
# print(f"Test loss: {results[0]}")
# print(f"Test MAE: {results[1]}")
# print(f"Test MSE: {results[2]}")
# print(f"Test MAPE: {results[3]}")

In [123]:
# model_Tc.fit(X_train_Tc, Y_train_Tc, epochs=50, batch_size=32)

In [124]:
# history = model_Tc.fit(
#     X_train_Tc, Y_train_Tc,
#     validation_data=(X_test_Tc, Y_test_Tc),
#     epochs=200,
#     batch_size=32,   
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=20, restore_best_weights=True),
#         tf.keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=5)
#     ]
# )

In [125]:
# results = model_Tc.evaluate(X_test_Tc, Y_test_Tc)
# print(f"Test loss: {results[0]}")
# print(f"Test MAE: {results[1]}")
# print(f"Test MSE: {results[2]}")
# print(f"Test MAPE: {results[3]}")

In [126]:
# model_Density.fit(X_train_D, Y_train_D, epochs = 25, batch_size=16)

In [127]:
# history = model_Density.fit(
#     X_train_D, Y_train_D,
#     validation_data=(X_test_D, Y_test_D),
#     epochs=200,
#     batch_size=16,   
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=20, restore_best_weights=True),
#         tf.keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=5)
#     ]
# )

In [128]:
# results = model_Density.evaluate(X_test_D, Y_test_D)
# print(f"Test loss: {results[0]}")
# print(f"Test MAE: {results[1]}")
# print(f"Test MSE: {results[2]}")
# print(f"Test MAPE: {results[3]}")

In [129]:
# model_Rg.fit(X_train_Rg, Y_train_Rg, epochs = 25, batch_size=16)

In [130]:
# history = model_Rg.fit(
#     X_train_Rg, Y_train_Rg,
#     validation_data=(X_test_Rg, Y_test_Rg),
#     epochs=200,
#     batch_size=16,   
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=20, restore_best_weights=True),
#         tf.keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=5)
#     ]
# )

In [131]:
# results = model_Rg.evaluate(X_test_Rg, Y_test_Rg)
# print(f"Test loss: {results[0]}")
# print(f"Test MAE: {results[1]}")
# print(f"Test MSE: {results[2]}")
# print(f"Test MAPE: {results[3]}")

In [132]:
Y_Tg_sub = model_Tg.predict(X_sub)
Y_Tg_sub

array([140.731   , 164.28232 ,  83.722275], dtype=float32)

In [133]:
Y_Tc_sub = model_Tc.predict(X_sub)
Y_Tc_sub

array([0.17581089, 0.2344305 , 0.26910666], dtype=float32)

In [134]:
Y_Density_sub = model_Density.predict(X_sub)
Y_Density_sub

array([1.2468625, 1.1135224, 1.0915009], dtype=float32)

In [135]:
Y_Rg_sub = model_Rg.predict(X_sub)
Y_Rg_sub

array([23.231552, 20.838667, 20.083252], dtype=float32)

In [136]:
submission = pd.DataFrame({
    "id": test_csv["id"],
    "Tg" : Y_Tg_sub.ravel(),
    "FFV" : Y_FFV_sub.ravel(),
    "Tc" : Y_Tc_sub.ravel(),
    "Density": Y_Density_sub.ravel(),
    "Rg" : Y_Rg_sub.ravel()
})
submission


,id,Tg,FFV,Tc,Density,Rg
0,1109053969,140.731003,0.368397,0.175811,1.246863,23.231552
1,1422188626,164.282318,0.376375,0.234431,1.113522,20.838667
2,2032016830,83.722275,0.350371,0.269107,1.091501,20.083252


In [137]:
submission.to_csv("submission.csv", index=False)
print("Done")

Done
